In [13]:
import gurobipy as gp
from gurobipy import *
import random as rd
import numpy as np

In [14]:
import openpyxl
wb1 = openpyxl.load_workbook('South2.xlsx', data_only = True)
st = wb1.sheetnames
st1 = []

Pl = []
for sheet_name in st:
    worksheet1 = wb1[sheet_name]
    st1.append(sheet_name)
    
    g1_value = worksheet1['G1'].value
    Pl.append(g1_value)

In [16]:
Routes=['North', 'South']
NoS=[9, len(st1)]

r = input('Choose a route between North and South: ')
#print(r)
for i in range(len(NoS)):
    if int(Routes.index(r))==i:
        S=NoS[i]
        print('Total number of stations in the route: ', S)
TD=[]
for (i) in range (S):
    d=int(input('\nNumber of days you want to spend in Station '+ st1[i] +'\n'))
    TD.append(d)
TD


Choose a route between North and South: South
Total number of stations in the route:  10

Number of days you want to spend in Station Vizianagaram Junction
1

Number of days you want to spend in Station Visakhapatnam Junction
1

Number of days you want to spend in Station Rajahmundry
1

Number of days you want to spend in Station Ongole
1

Number of days you want to spend in Station Kadapa
1

Number of days you want to spend in Station Guntakal Jn
1

Number of days you want to spend in Station Eluru
1

Number of days you want to spend in Station tirupati
1

Number of days you want to spend in Station Bhimavara Town
1

Number of days you want to spend in Station Anantapur
1


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)
  return subsets

In [22]:
ys = []

for i in range(S):
#     if TD[i] == 0:
#         y = 0
#         ys.append(y)
#     else:
#             y = 1
#             ys.append(y)
    print("station:  " + str(i+1))
    print(Pl[i]) #number of places in ith station
    L = [p for p in range(Pl[i])]
    L1=[p for p in range(Pl[i])]
    print(L)
    T = [b for b in range(TD[i])]
    p = Pl[i]
    d = TD[i]
    #time of travel between stations
#     tp=[[rd.uniform(0.1,3) for i in L]for j in L]
    
    for j in L:
        for i in L:
#             mat = np.random.uniform(low=0.1, high=3, size=(p,p))
            mat = np.random.randint(low=1, high=4, size=(p,p))

            mat = np.triu(mat, 1) + np.triu(mat, 1).T
            np.fill_diagonal(mat, 0)
    print(mat)

    

#     print(tp)
    
    #time of stay per station
#     ts = rd.randint(1,3, size = Pl[i])
#     print(ts)    
    
    ts = [rd.randint(1, 3) for i in range(p)]  # generate the random vector
    print(ts)
    m=gp.Model('Travel')
    
    #Variable
    xijt = m.addVars(L, L, T, vtype=GRB.BINARY, name ='xijt')
    
    #Objective Function
    obj_fn = sum(xijt[i,j,t] for t in T for i in L for j in L)
    m.setObjective(obj_fn, GRB.MAXIMIZE)
    
  
    #Constraints
    
#final constrints:  
    m.addConstrs((gp.quicksum(xijt[i,j,t] for t in T) <= 1 for i in L for j in L if i != 0 or j != 0))
    m.addConstrs(gp.quicksum(xijt[i,j,t] for t in T for i in L) <= 1 for j in L if i != 0 or j != 0)
    m.addConstrs(gp.quicksum(xijt[i,j,t] for t in T for j in L) <= 1 for i in L if i != 0 or j != 0)
    m.addConstrs(gp.quicksum(xijt[i,j,t] for i in L) == gp.quicksum(xijt[j,i,t] for i in L) for j in L for t in T)
    m.addConstrs(gp.quicksum(xijt[i,i,t] for t in T) == 0 for i in L)
    m.addConstrs(gp.quicksum(xijt[i,j,t]*mat[i][j]for i in L for j in L) +gp.quicksum(xijt[i,j,t]*ts[j] for j in L)<=12 for t in T)

#end
        
  
    
#     m.addConstrs(gp.quicksum(xijt[i,j,t] for i in L) <= 1 for j in L for t in T )
#     m.addConstrs(gp.quicksum(xijt[i,j,t] for j in L) <= 1 for i in L for t in T )
#     m.addConstrs(gp.quicksum(xijt[i,j,t] for t in T) <= p for i in L for j in L )
    
#     m.addConstrs(gp.quicksum(xijt[i,j,t] for i in L) <= 1 for j in L for t in T if i != 0)
#     m.addConstrs(gp.quicksum(xijt[i,j,t] for j in L) <= 1 for i in L for t in T if j != 0)


    m.addConstrs(gp.quicksum(xijt[0,j,t] for t in T) <= d for j in L)
    m.addConstrs(gp.quicksum(xijt[i,0,t] for t in T) <= d for i in L)
    
#     m.addConstrs(gp.quicksum(xijt[i,j,t]*mat[i][j] for t in T for i in L) <= d*12 - ts[j] for j in L if i != j)
#     m.addConstrs(gp.quicksum(xijt[i,j,t]*mat[i][j] for i in L for j in L) <= 12 - ts[j] for t in T if i != j)
    #m.addConstrs(gp.quicksum(xijt[i,j,t]*(mat[i][j] + ts[j] ) for i in L for j in L) <= 4 for t in T if i != j)

#     m.addConstrs(( sa[i] - sa[j] + mat[i][j] <= M(1-xijt[i,j,t])   for i in L for j in L )for t in T)

    
    print(d)

#     m.addConstrs(gp.quicksum(xijt[i,h,t] for i in L) - gp.quicksum(xijt[h,j,t] for j in L)== 0 for h in L if i != h and j !=h)

    
    
#     print("L is :  ")
#     print(L)
    #subtour elimination
#     for t in T:
#         for Su in powerset(L):
#     #             print(Su)
#     #             print(p)
#             if len(Su)>=1 and (len(Su)<p):
#                 sumt=0
#                 for i in Su:
#                         for j in L:
#                             if j not in Su:
#                                 sumt=sumt+xijt[i,j,t]
#                 m.addConstr(sumt, GRB.GREATER_EQUAL, 1)

#     sumcon = 0
#     for t in T:
#         for i in L:
#             for j in L:
#                 if i!= j and sumcon <=12 :
#                     sumcon = sumcon + xijt[j]*mat[i][j] + ts[j]
                    
#                 else:
#                     t = t +1


#     for t in T:
#         for i in L:
#                 for j in L:
#                     if i != j:
#                         m.addConstr(xijt[i, j, t] + xijt[j, i, t] <= 1)


    m.optimize()
    m.printAttr('X')
#     print(xijt)

station:  1
9
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[[0 1 2 2 2 1 3 1 1]
 [1 0 3 3 3 2 2 1 2]
 [2 3 0 2 3 2 1 1 1]
 [2 3 2 0 1 1 1 3 3]
 [2 3 3 1 0 1 1 1 2]
 [1 2 2 1 1 0 1 1 2]
 [3 2 1 1 1 1 0 1 2]
 [1 1 1 3 1 1 1 0 1]
 [1 2 1 3 2 2 2 1 0]]
[2, 1, 1, 1, 1, 2, 2, 3, 1]
1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 135 rows, 81 columns and 486 nonzeros
Model fingerprint: 0x8b26e812
Variable types: 0 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 107 rows and 9 columns
Presolve time: 0.00s
Presolved: 28 rows, 72 columns, 349 nonzeros
Variable types: 0 continuous, 72 integer (72 binary)
Found heu


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       7.0000000    7.00000  0.00%     -    0s

Explored 1 nodes (15 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 3: 7 6 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%

    Variable            X 
-------------------------
 xijt[0,6,0]            1 
 xijt[1,2,0]            1 
 xijt[2,3,0]            1 
 xijt[3,1,0]            1 
 xijt[4,5,0]            1 
 xijt[5,4,0]            1 
 xijt[6,0,0]            1 
station:  6
7
[0, 1, 2, 3, 4, 5, 6]
[[0 2 3 3 1 3 1]
 [2 0 3 3 2 1 2]
 [3 3 0 3 3 1 2]
 [3 3 3 0 1 1 1]
 [1 2 3 1 0 1 1]
 [3 1 1 1 1 0 3]
 [1 2 2 1 1 3 0]]
[2, 2, 2, 2, 2, 1, 3]
1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ry

Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%

    Variable            X 
-------------------------
 xijt[0,5,0]            1 
 xijt[1,0,0]            1 
 xijt[2,3,0]            1 
 xijt[3,2,0]            1 
 xijt[4,6,0]            1 
 xijt[5,4,0]            1 
 xijt[6,1,0]            1 


In [57]:
L


[0, 1, 2, 3, 4, 5, 6]

In [58]:
L1

[1, 2, 3, 4, 5, 6]